In [3]:
import pandas as pd
import numpy as np
from github import Auth
from github import Github
import json
import pickle
import os
import time
import shutil
from tqdm import tqdm

In [11]:
ACCESS_TOKEN = json.load(open("./config"))["access_token"]

REPO_METADATA_PATH = "./Data/BlockchainAppRepositories-Old.csv"

REPOSITORIES_BIN_DATA_PATH = "./Data/Repositories.bin"
ISSUE_DATA_PATH = "./Data/AllIssues-New.csv"

BACKUP_PATH = "./Data/Backup"

In [5]:
auth = Auth.Token(ACCESS_TOKEN)
github = Github(auth=auth)

In [6]:
def rate_limited(interval=600):
    last_executed = 0 

    def decorator(func):
        def wrapper(*args, **kwargs):
            nonlocal last_executed
            current_time = time.time()
            if current_time - last_executed >= interval:
                last_executed = current_time
                return func(*args, **kwargs)
            else:
                pass
        return wrapper
    return decorator

@rate_limited(120)
def backup(paths):
    if not os.path.isdir(BACKUP_PATH):
        os.mkdir(BACKUP_PATH)
    for path in paths:
        if os.path.exists(path):
            shutil.copyfile(path, BACKUP_PATH + "/" + os.path.basename(path) + time.strftime("%Y%m%d-%H%M%S"))

class Repository:
    def __init__(self, github_client, repo_url):
        self.github_client = github_client
        self.identifier = self.parse_repo_identifier(repo_url)
        self.issues = []
        self.issue_comments = {}

    def parse_repo_identifier(self, repo_url):
        owner = repo_url.split("/")[-2]
        repo_name = repo_url.split("/")[-1]
    
        return owner + "/" + repo_name
    
    def get_identifier(self):
        return self.identifier

    def collect_info(self):
        self.repo = self.github_client.get_repo(self.identifier)
        self.set_issues(self.repo.get_issues(state="all"))

    def set_issues(self, issues):
        self.issues = issues
        for issue in self.issues:
            self.set_comments(issue.number, issue.get_comments())

    def set_comments(self, issue_id, comments):
        self.issue_comments[issue_id] = comments

    def get_issues(self):
        issues = []
        for issue in self.issues:
            issues.append({
                "Repository": self.identifier,
                "IssueId": issue.number,
                "Title": issue.title,
                "Body": issue.body,
                "State": issue.state,
                "Label": ",".join([label.name for label in issue.labels]),
                "CreatedAt": issue.created_at,
                "ClosedAt": issue.closed_at
            })
        return pd.DataFrame(issues)

    ## Not necessary and takes too much time
    def get_comments(self):
        comments = []
        for issue_id in self.issue_comments.keys():
            for comment in self.issue_comments[issue_id]:
                comments.append({
                    "Repository": self.identifier,
                    "IssueId": issue_id,
                    "CommentId": comment.id,
                    "Comment": comment.body
                })
        return pd.DataFrame(comments)

In [7]:
repositories = []

processed_repos = []
if os.path.exists(ISSUE_DATA_PATH):
    processed_repos = pd.read_csv(ISSUE_DATA_PATH)["Repository"].unique().tolist()

df = pd.read_csv(REPO_METADATA_PATH)

for url in tqdm(df["repo_url"].to_list()):
    try:
        repo = Repository(github, url)
        if not repo.get_identifier() in processed_repos:
            repo.collect_info()
            repositories.append(repo)

            if os.path.exists(ISSUE_DATA_PATH):
                issues = pd.read_csv(ISSUE_DATA_PATH)
                issues = pd.concat([issues, repo.get_issues()], ignore_index=True, sort=False)
            else:
                issues = repo.get_issues()
            
            backup([ISSUE_DATA_PATH, REPOSITORIES_BIN_DATA_PATH])

            issues.to_csv(ISSUE_DATA_PATH, index=False)
            #pickle.dump(repositories,  open(REPOSITORIES_BIN_DATA_PATH, 'wb'))
    except Exception as e:
        print("Error: ", e)

  2%|▏         | 21/991 [01:52<34:29,  2.13s/it]  

Error:  404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest", "status": "404"}


  2%|▏         | 23/991 [01:53<21:14,  1.32s/it]

Error:  404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest", "status": "404"}


  2%|▏         | 24/991 [01:53<16:25,  1.02s/it]

...

 85%|████████▍ | 841/991 [2:14:16<26:54, 10.76s/it]

Error:  404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest", "status": "404"}


 97%|█████████▋| 962/991 [2:41:51<10:20, 21.38s/it]  

Error:  404 {"message": "Not Found", "documentation_url": "https://docs.github.com/rest", "status": "404"}


100%|██████████| 991/991 [2:49:44<00:00, 10.28s/it]


In [8]:
repo = github.get_repo("opexdev/core")

In [14]:
df

,url,name,description,language,Category,blockchain
0,https://api.github.com/repos/askmike/gekko,gekko,A bitcoin trading bot written in node,JavaScript,Crypto,bitcoin
1,https://api.github.com/repos/butor/blackbird,blackbird,Blackbird Bitcoin Arbitrage: a long/short mark...,C++,Crypto,bitcoin
2,https://api.github.com/repos/MetaMask/metamask...,metamask-extension,:globe_with_meridians: :electric_plug: metamas...,JavaScript,Crypto,ethereum
3,https://api.github.com/repos/bitcoin-wallet/bi...,bitcoin-wallet,Bitcoin Wallet app for your Android device. St...,Java,Crypto,bitcoin
4,https://api.github.com/repos/yasinkuyu/binance...,binance-trader,Bitcoin Trading Bot for Binance (Experimental),NaN,Crypto,multi
...,...,...,...,...,...,...
1189,https://api.github.com/repos/simplestaking/tez...,tezos-wallet,NaN,NaN,Crypto,tezos
1190,https://api.github.com/repos/SJSU272LabF17/Org...,OrganDonation-Chain,Organ Waitlist ledger on a blockchain,JavaScript,Others,hyperledger fabric
1191,https://api.github.com/repos/dfoderick/bitshovel,bitshovel,Bitcoin message bus bridge to internal message...,NaN,Others,bitcoin
1192,https://api.github.com/repos/ampleforth/market...,market-oracle,Set of smart contracts on Ethereum deal with e...,JavaScript,Crypto,ethereum


In [17]:

df = pd.read_csv(REPO_METADATA_PATH)
updated_at = []
for url in tqdm(df["url"].to_list()):
    owner = url.split("/")[-2]
    repo_name = url.split("/")[-1]
    try:
        repo = github.get_repo(f"{owner}/{repo_name}")
        updated_at.append(repo.pushed_at)
    except:
        updated_at.append(None)

100%|██████████| 1194/1194 [08:33<00:00,  2.32it/s]


In [ ]:
df["updated_at"] = updated_at
df = df[df["updated_at"] >= "2024-02-21"]
df.to_csv("./Data/BlockchainAppRepositories-Old-Actives.csv", index=False)

In [38]:
actve_repo = [url.split("/")[-2] + "/" + url.split("/")[-1] for url in df["url"].to_list()]

In [ ]:
issues = pd.read_csv("./Data/AllIssues-Old.csv")
issues = issues[issues["Repository"].isin(actve_repo)]
issues.to_csv("./Data/AllIssues-Old-Actives.csv", index=False)

<ipython-input-39-2ab6b713f0ab>:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  issues = pd.read_csv("./Data/AllIssues-Old.csv")


,Repository,IssueId,Title,Body,State,Label,CreatedAt,ClosedAt
0,askmike/gekko,2892,"Sqlite issue, Gekko no finding sqlite on live ...","**Note: this is the technical bug tracker, ple...",closed,NaN,2020-02-14 19:59:40+00:00,2020-02-16 14:15:40+00:00
1,askmike/gekko,2891,Unable to see the functionality of any page,![Screenshot (2)](https://user-images.githubus...,closed,NaN,2020-02-13 04:46:29+00:00,2020-02-16 14:15:51+00:00
2,askmike/gekko,2890,Error: non-error thrown: ERROR: Talib is not e...,"**Note: this is the technical bug tracker, ple...",closed,NaN,2020-01-28 15:51:00+00:00,2020-02-16 14:16:37+00:00
3,askmike/gekko,2889,Update dependencies for Kraken import,I had to add/update these packages to be able ...,closed,NaN,2020-01-26 19:33:56+00:00,2020-02-16 14:18:45+00:00
4,askmike/gekko,2888,Cannot trade on coinfalcon,"**Note: this is the technical bug tracker, ple...",closed,NaN,2020-01-21 23:44:54+00:00,2020-02-16 14:16:08+00:00
...,...,...,...,...,...,...,...,...
332670,thebeetoken/beenest-web,5,Upgrade webpack-dev-server,https://www.npmjs.com/advisories/725\r\n\r\n##...,closed,NaN,2019-01-02 20:34:21+00:00,2019-01-02 23:40:14+00:00
332671,thebeetoken/beenest-web,4,Host onboarding redirect,## Description\r\nMinor Host onboarding tweaks...,closed,NaN,2019-01-02 18:45:37+00:00,2019-01-02 19:50:50+00:00
332672,thebeetoken/beenest-web,3,admin ux tweaks,"## Description\r\nRemoves unneeded CSS, joins ...",closed,NaN,2019-01-02 18:32:57+00:00,2019-01-02 19:37:39+00:00
332673,thebeetoken/beenest-web,2,Deploy,fix for photo and country code saving,closed,NaN,2018-12-30 03:56:00+00:00,2018-12-30 04:17:11+00:00
